In [24]:
import pandas as pd
import sqlite3
import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

from sqlalchemy import create_engine

In [50]:
#connection to a PostgreSQL local database
# ===========================================

# user_name_var = input('What is the username of your database? ')
host_var = input('What is the host address of your database?')
pw = input("Enter the database password: ")
# db_name = input("Enter the name of the database: ")

string_output = "postgresql://postgres"+':'+pw+'@'+host_var+":5432"+'/earthquake_db'
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

Connected to database...


In [51]:
#create and connect to a sqlite3 db
conn = sqlite3.connect(r'/Users/rosaicelaroman/Desktop/portfolio_work/earthquake_modeling/Database/db/earthquake_db')

Adding 1 month of data

In [52]:
page_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

df = pd.read_csv(page_url)
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-05-24T03:58:35.650Z,32.179167,-115.821000,0.42,2.07,ml,13.0,107.0,0.20460,0.17,...,2022-05-24T04:02:17.749Z,"50km SSW of Progreso, B.C., MX",earthquake,0.52,31.61,0.182,26.0,automatic,ci,ci
1,2022-05-24T03:52:14.260Z,36.414501,-120.993835,6.74,1.88,md,24.0,109.0,0.02278,0.16,...,2022-05-24T03:55:07.210Z,"19km SE of Pinnacles, CA",earthquake,0.45,0.72,0.110,14.0,automatic,nc,nc
2,2022-05-24T03:37:55.783Z,59.961600,-152.287000,67.80,2.10,ml,NaN,NaN,NaN,0.59,...,2022-05-24T03:49:48.431Z,"31 km W of Happy Valley, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
3,2022-05-24T03:10:25.890Z,38.817833,-122.759331,1.63,0.45,md,9.0,130.0,0.01509,0.01,...,2022-05-24T03:40:15.362Z,"3km W of Cobb, CA",earthquake,0.42,0.70,0.400,2.0,automatic,nc,nc
4,2022-05-24T03:09:58.740Z,36.095667,-117.792667,8.25,1.26,ml,21.0,53.0,0.05491,0.16,...,2022-05-24T03:13:44.586Z,"15km ENE of Coso Junction, CA",earthquake,0.29,0.54,0.181,16.0,automatic,ci,ci
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8818,2022-04-24T04:24:07.109Z,57.315700,-155.514800,80.60,1.80,ml,NaN,NaN,NaN,0.22,...,2022-04-24T04:28:50.319Z,"69 km WSW of Karluk, Alaska",earthquake,NaN,1.70,NaN,NaN,automatic,ak,ak
8819,2022-04-24T04:14:08.355Z,26.836900,126.541700,10.00,4.50,mb,NaN,76.0,1.54600,1.09,...,2022-05-04T02:35:01.040Z,"132 km WNW of Naha, Japan",earthquake,5.00,1.90,0.157,13.0,reviewed,us,us
8820,2022-04-24T04:14:04.933Z,35.155600,26.970500,10.00,4.70,mb,NaN,53.0,0.42000,1.28,...,2022-05-20T01:10:00.040Z,"29 km S of Fry, Greece",earthquake,5.30,1.80,0.064,77.0,reviewed,us,us
8821,2022-04-24T04:10:26.090Z,19.227501,-155.385162,32.82,1.79,md,35.0,150.0,NaN,0.10,...,2022-04-24T04:13:34.580Z,"10 km ENE of Pāhala, Hawaii",earthquake,0.60,0.98,1.240,8.0,automatic,hv,hv


In [53]:
#append the new data from USGS to both databases
#=============
#PostgreSQL
df.to_sql('earthquake_raw', engine, if_exists='append', index = False)
#=============
#SQLite
cur = conn.cursor()
df.to_sql('earthquake_raw', conn, if_exists='append', index=False)

8823

Load from database to update with new rows and remove duplicate rows

In [54]:
#Dataframe from postgresql
model_df = pd.read_sql_table(
    "earthquake_raw",
    con=engine
)
model_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.8750,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.1190,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62542,2022-04-24T04:24:07.109Z,57.315700,-155.514800,80.60,1.80,ml,NaN,NaN,NaN,0.2200,...,2022-04-24T04:28:50.319Z,"69 km WSW of Karluk, Alaska",earthquake,NaN,1.70,NaN,NaN,automatic,ak,ak
62543,2022-04-24T04:14:08.355Z,26.836900,126.541700,10.00,4.50,mb,NaN,76.00,1.5460,1.0900,...,2022-05-04T02:35:01.040Z,"132 km WNW of Naha, Japan",earthquake,5.00,1.90,0.157,13.0,reviewed,us,us
62544,2022-04-24T04:14:04.933Z,35.155600,26.970500,10.00,4.70,mb,NaN,53.00,0.4200,1.2800,...,2022-05-20T01:10:00.040Z,"29 km S of Fry, Greece",earthquake,5.30,1.80,0.064,77.0,reviewed,us,us
62545,2022-04-24T04:10:26.090Z,19.227501,-155.385162,32.82,1.79,md,35.0,150.00,NaN,0.1000,...,2022-04-24T04:13:34.580Z,"10 km ENE of Pāhala, Hawaii",earthquake,0.60,0.98,1.240,8.0,automatic,hv,hv


In [55]:
#dataframe from SQLite
model_df2 = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
model_df2

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.8750,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.1190,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62542,2022-04-24T04:24:07.109Z,57.315700,-155.514800,80.60,1.80,ml,NaN,NaN,NaN,0.2200,...,2022-04-24T04:28:50.319Z,"69 km WSW of Karluk, Alaska",earthquake,NaN,1.70,NaN,NaN,automatic,ak,ak
62543,2022-04-24T04:14:08.355Z,26.836900,126.541700,10.00,4.50,mb,NaN,76.00,1.5460,1.0900,...,2022-05-04T02:35:01.040Z,"132 km WNW of Naha, Japan",earthquake,5.00,1.90,0.157,13.0,reviewed,us,us
62544,2022-04-24T04:14:04.933Z,35.155600,26.970500,10.00,4.70,mb,NaN,53.00,0.4200,1.2800,...,2022-05-20T01:10:00.040Z,"29 km S of Fry, Greece",earthquake,5.30,1.80,0.064,77.0,reviewed,us,us
62545,2022-04-24T04:10:26.090Z,19.227501,-155.385162,32.82,1.79,md,35.0,150.00,NaN,0.1000,...,2022-04-24T04:13:34.580Z,"10 km ENE of Pāhala, Hawaii",earthquake,0.60,0.98,1.240,8.0,automatic,hv,hv


In [56]:
#check for duplicates in postgres data
boolean = model_df.duplicated(subset=['id']).any()
print(boolean)

True


In [57]:
#check for duplicates in sqlite data
boolean = model_df2.duplicated(subset=['id']).any()
print(boolean)

True


In [58]:
#drop duplicates in postgres data
model_df.drop_duplicates(subset=['id'], inplace=True)
print(model_df)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
53720  2022-04-24T04:01:45.641Z  46.155600  152.598700   10.00  4.50      mb   
53721  2022-04-24T04:00:21.800Z  33.886500 -118.289167   12.29  0.94      ml   
53722  2022-04-24T04:00:03.173Z  46.253500  152.546600   29.49  4.30      mb   
53723  2022-04-24T04:00:02.810Z  33.923667 -118.282500   14.59  1.08      ml   
53724  2022-05-24T03:58:35.930Z  32.166000 -115.810997    7.13  2.17      ml   

        nst     gap     dmin     rms  .

In [59]:
#drop duplicates in sqlite data
model_df2.drop_duplicates(subset=['id'], inplace=True)
print(model_df2)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
53719  2022-04-24T04:01:45.641Z  46.155600  152.598700   10.00  4.50      mb   
53720  2022-04-24T04:00:21.800Z  33.886500 -118.289167   12.29  0.94      ml   
53721  2022-04-24T04:00:03.173Z  46.253500  152.546600   29.49  4.30      mb   
53722  2022-04-24T04:00:02.810Z  33.923667 -118.282500   14.59  1.08      ml   
53723  2022-05-24T03:58:35.930Z  32.166000 -115.810997    7.13  2.17      ml   

        nst     gap     dmin     rms  .

In [60]:
#replace the table in postgres with one without duplicates
model_df.to_sql('earthquake_raw', engine, if_exists='replace', index = False)

724

In [61]:
#replace the table in sqlite with one without duplicates
model_df2.to_sql('earthquake_raw', conn, if_exists='replace', index = False)

53724

In [62]:
#copy data as csv as backup method
model_df.to_csv("../Database/csv/earthquake_raw.csv",index=False)

In [63]:
# Import API key
import sys
weather_api_key = "YOUR KEY HERE!"

if weather_api_key == "YOUR KEY HERE!":
    sys.path.append(r"/Users/rosaicelaroman/Desktop/Data_BootCamp/APIs/config_keys/")
    from api_keys_v2 import weather_api_key

In [39]:
#create new dataframe for locatiions
# location_df = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
# location_df

In [40]:
#create new df with only necessary columns
# cleaned_df = location_df.drop(columns=['nst','rms','magNst','net', 'time', 'magType', 'gap','dmin', 'updated', 'place', 'type', 'horizontalError', 'depthError', 'magError', 'status', 'locationSource', 'magSource'])
# cleaned_df

http://api.openweathermap.org/geo/1.0/reverse?lat={lat}&lon={lon}&limit={limit}&appid={API key}

In [41]:
#create new columns for data from API
# cleaned_df['Location Name']=""
# cleaned_df['City']=""
# cleaned_df['State']=""
# cleaned_df['Country']=""
# cleaned_df

In [64]:
location_df = pd.read_sql_query("SELECT * FROM earthquake_raw_locations", conn)
location_df

,latitude,longitude,depth,mag,id,Location Name,City,State,Country
0,37.384000,-117.107500,1.10,1.30,nn00831163,Nye County,Nye County,Nevada,US
1,38.181000,-117.818000,7.30,1.40,nn00831161,Esmeralda County,Esmeralda County,Nevada,US
2,61.340700,-149.885200,30.00,1.50,ak0229xypaw,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
3,62.870300,-150.762500,111.30,2.10,ak0229xwriz,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
4,64.014500,-148.765700,1.90,1.50,ak0229xvr66,Denali,Denali,Alaska,US
...,...,...,...,...,...,...,...,...,...
38781,35.322667,-117.783333,4.70,1.09,ci39983311,Kern County,Kern County,California,US
38782,63.390200,-151.341800,0.00,1.80,ak02246jyds7,Denali,Denali,Alaska,US
38783,60.602200,-150.643000,19.40,2.10,ak02246jxbq8,Kenai Peninsula,Kenai Peninsula,Alaska,US
38784,35.965500,-117.677167,2.37,0.79,ci39983287,Inyo County,Inyo County,California,US


In [65]:
base_url = "http://api.openweathermap.org/geo/1.0/reverse?"



http://api.openweathermap.org/geo/1.0/reverse?lat=51.5098&lon=-0.1180&limit=5&appid={API key}

In [49]:
# response = requests.get(base_url+"lat="+'37.3840'+'&lon='+'-117.107500'+'&limit='+'1'+'&appid=' + weather_api_key).json()
# print(response[0]['country'])

US


In [66]:
import requests
import time

for index, row in location_df.iterrows():
    # count+=1
    lat = row['latitude']
    lon = row['longitude']
    loc_found = []

    if row['Country'] != "":
        continue

    print(f'Retrieving results for Index {index}: {lat}')
    try:
        response = requests.get(base_url+"lat="+str(lat)+'&lon='+str(lon)+'&limit='+'1'+'&appid=' + weather_api_key).json()
        # response = requests.get(base_url+city+"&units="+units+'&appid=' + weather_api_key).json()
        location_df.loc[index, 'Location Name'] = response[0]['name']
        location_df.loc[index, 'City'] = response[0]['local_names']['en']
        location_df.loc[index, 'State'] = response[0]['state']
        location_df.loc[index, 'Country'] = response[0]['country']
        loc_found.append(lat)

        print(f'Found lat: {lat}')
    except:
        print(f"Missing lat: {lat}... Skipping")
        location_df.loc[index, 'Location Name'] = float('NaN')
        location_df.loc[index, 'City'] = float('NaN')
        location_df.loc[index, 'State'] = float('NaN')
        location_df.loc[index, 'Country'] = float('NaN')
    print(f'--------------------')
    time.sleep(2)


In [46]:
location_df

,latitude,longitude,depth,mag,id,Location Name,City,State,Country
0,37.384000,-117.107500,1.10,1.30,nn00831163,Nye County,Nye County,Nevada,US
1,38.181000,-117.818000,7.30,1.40,nn00831161,Esmeralda County,Esmeralda County,Nevada,US
2,61.340700,-149.885200,30.00,1.50,ak0229xypaw,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
3,62.870300,-150.762500,111.30,2.10,ak0229xwriz,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
4,64.014500,-148.765700,1.90,1.50,ak0229xvr66,Denali,Denali,Alaska,US
...,...,...,...,...,...,...,...,...,...
38781,35.322667,-117.783333,4.70,1.09,ci39983311,Kern County,Kern County,California,US
38782,63.390200,-151.341800,0.00,1.80,ak02246jyds7,Denali,Denali,Alaska,US
38783,60.602200,-150.643000,19.40,2.10,ak02246jxbq8,Kenai Peninsula,Kenai Peninsula,Alaska,US
38784,35.965500,-117.677167,2.37,0.79,ci39983287,Inyo County,Inyo County,California,US


In [47]:
#replace the table in sqlite with one without duplicates
location_df.to_sql('earthquake_raw_locations', conn, if_exists='replace', index = False)

#replace the table in postgres with one without duplicates
location_df.to_sql('earthquake_raw_locations', engine, if_exists='replace', index = False)

786

In [48]:
# Close connection to SQLite database
conn.close()
connection.close()